# 导入模块

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

In [50]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [51]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [52]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.has_actual, om.relet_status, om.order_finish_date, om.overdue, tprm.rembursement_status, tprm.purchase_amount
,tol.delivery_province_name, tprm.contract_price, tprm.rent_received, tprm.all_rental, tprm.unearned_rent_revenue
, tprm.unrecovered_cost, tprm.overdue_type
from  db_digua_business.t_postlease_receivables_monitoring   tprm
left join db_digua_business.t_order  om on tprm.order_id=om.id
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
left join db_digua_business.t_merchant tmer on tmer.id = om.merchant_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
AND tmer.shop_type !=2  -- 剔除商家

'''

df_order = query(sql1)
df_order.shape

(24072, 42)

# 提取订单风控数据

# 合并数据

In [53]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(24072)

# 处理日期

In [54]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', 'has_actual',
       'relet_status', 'order_finish_date', 'overdue', 'rembursement_status',
       'purchase_amount', 'delivery_province_name', 'contract_price',
       'rent_received', 'all_rental', 'unearned_rent_revenue',
       'unrecovered_cost', 'overdue_type'],
      dtype='object')

In [55]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df["下单月份"] = df["create_time"].dt.strftime('%Y-%m')
df.shape


(24072, 45)

In [56]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()


拒绝理由
                                                                                     23108
命中云商分低于515强拒(芝麻综合风险等级0豁免)                                                              543
命中火眸分<550,并且为搜索渠道(自有模型免审豁免)                                                             17
冰鉴火眸分<500命中强拒(芝麻综合风险等级0豁免)                                                              16
命中青云分<500,并且为搜索渠道(自有模型免审豁免)                                                              6
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 命中青云分<500,并且为搜索渠道(自有模型免审豁免)                                 6
命中融安分低于680强拒(芝麻综合风险等级0豁免)                                                                5
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 履约历史等级1且为搜索渠道(自有模型免审豁免)                                     4
命中云商分低于515强拒(芝麻综合风险等级0豁免), 命中融安分低于680强拒(芝麻综合风险等级0豁免)                                     2
命中云商分低于515强拒(芝麻综合风险等级0豁免), 冰鉴火眸分<500命中强拒(芝麻综合风险等级0豁免)                                    2
综合风险等级为3且非免押, 蚁盾分>=80                                                                

# 处理备注信息

In [57]:
df = df[df['sku_attributes'].notnull()]


In [58]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].astype(str).str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].astype(str).str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [59]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
1    14207
0     9865
Name: count, dtype: int64

# 渠道归属

In [60]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                   14793
搜索渠道                    4811
未知渠道                    1151
租机宝                      659
单人聊天会话中的小程序消息卡片（分享）      396
                       ...  
618双重狂欢                    1
糖果MAU04                    1
邦道-自定义                     1
邦道-销售线索                    1
邦客惠乘车码                     1
Name: count, Length: 61, dtype: int64

# 订单去重

In [61]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', 'has_actual',
       'relet_status', 'order_finish_date', 'overdue', 'rembursement_status',
       'purchase_amount', 'delivery_province_name', 'contract_price',
       'rent_received', 'all_rental', 'unearned_rent_revenue',
       'unrecovered_cost', 'overdue_type', '下单日期', '月份', '下单月份', '拒绝理由',
       '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否', '是否进行预授权',
       '颜色', '内存', '进件', '来源渠道', '免审', '归属渠道'],
      dtype='object')

In [62]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [63]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [64]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
人审拒绝    24047
机审强拒        9
Name: count, dtype: int64

# 保留商家的数据

In [65]:
df_contain = df.copy()

# 剔除商家数据

In [66]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)


# 去重订单明细导出

In [67]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-10-31'

# 导入服务订单

In [68]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(24053, 66)

In [69]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)



# 导入出库台账数据

In [70]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# wb = pd.ExcelFile(f_path_ck)  
# df_ck = pd.read_excel(wb, sheet_name="2023")
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")   #,skiprows=0 header 
df_ck.shape

(20732, 18)

In [71]:
df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

(19391, 97)

In [72]:
dfck.columns

Index(['日期', '订单号', '类目', '型号', '内存_x', '颜色_x', '平台套餐', '租金', '期数', '已付金额',
       '订单归属', '出租方', '备注', '快递单号', '供应商', '采购金额', '碎屏保', '串码', '溢价费订单', '单号',
       '溢价费', 'create_date', 'create_time', 'order_id', 'order_number',
       'all_money', 'status', 'status2', 'channel_type', 'sku_attributes',
       'product_name', 'new_actual_money', 'back_type', 'user_mobile',
       'true_name', 'id_card_num', 'total_describes', 'decision_result',
       'cancel_reason', 'trace_id', 'rejected', 'result', 'channel_name',
       'activity_name', 'merchant_id', 'merchant_name',
       'total_freeze_fund_amount', 'buy_service_product', 'service_status',
       'order_method', 'has_actual', 'relet_status', 'order_finish_date',
       'overdue', 'rembursement_status', 'purchase_amount',
       'delivery_province_name', 'contract_price', 'rent_received',
       'all_rental', 'unearned_rent_revenue', 'unrecovered_cost',
       'overdue_type', '下单日期', '月份', '下单月份', '拒绝理由', '取消原因', '电审拒绝原因',
       '无

In [73]:
dfck[dfck["status2"]=="已退款"]

dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)


In [74]:
df['result'].value_counts()

result
【风控复核】建议商户对租户进行电审，如：如 1.通过核实用户的工作情况和累计贷款金额等，判断偿还能力；2.核实常驻地址；3.增加紧急联系人。为保证订单流程规范，发货前请注意核实租户身份证正反面图片是否正常上传，以便在订单发生坏账时更有利于起诉。提醒租户需要下单人本人带身份证签收。    15965
【风控通过】该用户风控资质较好，建议商户可以优先审核发货。为保证订单流程规范，发货前请注意核实租户身份证正反面图片是否正常上传，以便在订单发生坏账时更有利于起诉。提醒租户需要下单人本人带身份证签收。                                              7125
【风控复核】建议商户对租户进行电审，如：1.通过核实用户的工作情况和累计贷款金额等，判断偿还能力；2.核实常驻地址；3.增加紧急联系人。为保证订单流程规范，发货前请注意核实租户身份证正反面图片是否正常上传，以便在订单发生坏账时更有利于起诉。提醒租户需要下单人本人带身份证签收。        596
                                                                                                                                                   11
【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。                                                                                                                   9
【优质订单】免人审                                                                                                                                           1
数据校验通过                                                                                       

In [75]:
def M_yq(df, keys):
    df_zt = df[df["unrecovered_cost"]>0] 
    df_zt['M0'] = np.where(df_zt.overdue_type=='M0',df_zt["unrecovered_cost"],0)
    df_zt['M1'] = np.where(df_zt.overdue_type=='M1',df_zt["unrecovered_cost"],0)
    df_zt['M2'] = np.where(df_zt.overdue_type=='M2',df_zt["unrecovered_cost"],0)
    df_zt['M3'] = np.where(df_zt.overdue_type=='M3',df_zt["unrecovered_cost"],0)
    df_zt['M4'] = np.where(df_zt.overdue_type=='M4',df_zt["unrecovered_cost"],0)
    df_zt['M5'] = np.where(df_zt.overdue_type=='M5',df_zt["unrecovered_cost"],0)
    df_zt['M6'] = np.where(df_zt.overdue_type=='M6',df_zt["unrecovered_cost"],0)
    dfyq = df_zt.groupby([keys]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
    dfyq=dfyq.fillna(0)
    dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
    dfyq1=pd.merge(dfyq,df.groupby([keys]).agg({"purchase_amount":np.sum}),left_on=keys,right_on=keys,how="left")
    dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
    dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["purchase_amount"]
    dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
    dfyq1["M2+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"])/dfyq1["purchase_amount"]
    dfyq1["M2+逾期率"]=dfyq1["M2+逾期率"].apply(lambda x:format(x,".2%"))
    dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["purchase_amount"]
    dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
    dfyq1 = dfyq1[['M0','M1','M2','M3','M6', 'purchase_amount', 'M1+逾期率','M2+逾期率','M3+逾期率']]

    return dfyq1

In [76]:
M_yq(df, '下单日期')

,M0,M1,M2,M3,M6,purchase_amount,M1+逾期率,M2+逾期率,M3+逾期率
下单日期,,,,,,,,,
2022-07-18 00:00:00,0.00,0.00,0.0,0.00,1491.76,6.678000e+03,22.34%,22.34%,22.34%
2022-07-20 00:00:00,0.00,0.00,0.0,0.00,12027.14,1.989900e+04,60.44%,60.44%,60.44%
2022-07-27 00:00:00,0.00,0.00,0.0,0.00,6958.64,1.356000e+04,51.32%,51.32%,51.32%
2022-07-28 00:00:00,0.00,0.00,0.0,0.00,5552.53,2.621500e+04,21.18%,21.18%,21.18%
2022-07-31 00:00:00,0.00,0.00,0.0,0.00,11966.17,1.669000e+04,71.70%,71.70%,71.70%
...,...,...,...,...,...,...,...,...,...
2024-10-27 00:00:00,0.00,0.00,0.0,0.00,0.00,3.059520e+05,0.00%,0.00%,0.00%
2024-10-28 00:00:00,0.00,0.00,0.0,0.00,0.00,4.916790e+05,0.00%,0.00%,0.00%
2024-10-29 00:00:00,0.00,0.00,0.0,0.00,0.00,2.877720e+05,0.00%,0.00%,0.00%


# 到期订单

In [77]:
# df = df[(df['下单月份']>='2022-05')&(df['下单月份']<='2024-09')]

In [80]:
# 定义一个函数来计算年龄
def calculate_age(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 应用函数到 DataFrame
df['age'] = df.apply(lambda row: calculate_age(row['id_card_num'], row['create_time']), axis=1)
df = df[(df['purchase_amount']>0)]

# 性别分布

In [81]:
# 定义一个函数来判断性别
def get_gender(id_card):
    gender_digit = int(id_card[-2])  # 第17位数字
    return '男' if gender_digit % 2 != 0 else '女'

# 应用函数到 DataFrame
df['性别'] = df['id_card_num'].apply(get_gender)
df_sex_M = M_yq(df, '性别')

In [82]:
df_sex = df.groupby('性别').agg({'是否出库': 'sum', 'new_actual_money': 'sum'})
df_sex.rename(columns={'是否出库': '出库', 'new_actual_money': '买断价'}, inplace=True)
df_sex


,出库,买断价
性别,,
女,6356,7.397639e+07
男,17086,1.871133e+08


In [83]:
df_tc = df[df['status2']!='已完成']
df_sex_group = df_tc.groupby('性别').agg({'是否出库': 'sum', 'new_actual_money': 'sum'})
df_sex_group.rename(columns={'是否出库': '出库(移除已完成)', 'new_actual_money': '买断价(移除已完成)'}, inplace=True)
df_sexs = pd.merge(df_sex, df_sex_group, on='性别', how='outer')
df_sexs

,出库,买断价,出库(移除已完成),买断价(移除已完成)
性别,,,,
女,6356,7.397639e+07,5774,6.781042e+07
男,17086,1.871133e+08,14822,1.640232e+08


In [84]:
df_sex_M

,M0,M1,M2,M3,M6,purchase_amount,M1+逾期率,M2+逾期率,M3+逾期率
性别,,,,,,,,,
女,1065974.83,518903.38,418717.67,794595.46,1505364.38,4.853662e+07,6.67%,5.60%,4.74%
男,2831402.80,1548159.43,1166905.53,2898371.70,8026754.42,1.231612e+08,11.08%,9.82%,8.87%
总计,3897377.63,2067062.81,1585623.20,3692967.16,9532118.80,1.716979e+08,9.83%,8.63%,7.70%


In [85]:
df_sex_Ms = df_sex_M[['M1+逾期率', 'M2+逾期率', 'M3+逾期率']]
df_sexs_group = pd.merge(df_sexs, df_sex_Ms[:-1], on='性别', how='outer')
df_sexs_group[['出库', '出库', '买断价', '出库(移除已完成)', '出库(移除已完成)', '买断价(移除已完成)', 'M1+逾期率', 'M2+逾期率', 'M3+逾期率']]

,出库,出库,买断价,出库(移除已完成),出库(移除已完成),买断价(移除已完成),M1+逾期率,M2+逾期率,M3+逾期率
性别,,,,,,,,,
女,6356,6356,7.397639e+07,5774,5774,6.781042e+07,6.67%,5.60%,4.74%
男,17086,17086,1.871133e+08,14822,14822,1.640232e+08,11.08%,9.82%,8.87%


# 年龄分布

In [86]:

# 定义年龄区间
age_bins = [0, 22, 30, 40, 55, float('inf')]
age_labels = ['(0,22]', '(22,30]', '(30,40]', '(40,55]', '[55+)']


# 使用 pd.cut 设置年龄区间
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=False)

df_age_M = M_yq(df, 'age_group')

df_age = df.groupby('age_group').agg({'是否出库': 'sum', 'new_actual_money': 'sum'})

df_age.rename(columns={'是否出库': '出库', 'new_actual_money': '买断价'}, inplace=True)

In [87]:
df_tc = df[df['status2']!='已完成']
df_age_tc = df_tc.groupby('age_group').agg({'是否出库': 'sum', 'new_actual_money': 'sum'})
df_age_tc.rename(columns={'是否出库': '出库(移除已完成)', 'new_actual_money': '买断价(移除已完成)'}, inplace=True)
df_ages = pd.merge(df_age, df_age_tc, on='age_group', how='outer')

In [88]:
df_age_M

,M0,M1,M2,M3,M6,purchase_amount,M1+逾期率,M2+逾期率,M3+逾期率
age_group,,,,,,,,,
"(0,22]",938826.47,382090.16,302903.83,599413.95,2296666.81,3.776853e+07,9.48%,8.47%,7.67%
"(22,30]",1568362.64,885089.31,684467.76,1864729.45,4301496.87,7.397998e+07,10.46%,9.26%,8.33%
"(30,40]",1138901.36,690479.48,445724.78,1031408.68,2312485.61,4.775429e+07,9.38%,7.94%,7.00%
"(40,55]",251287.16,109403.86,152526.83,197415.08,621469.51,1.219506e+07,8.86%,7.97%,6.71%
[55+),0.00,0.00,0.00,0.00,0.00,0.000000e+00,nan%,nan%,nan%
总计,3897377.63,2067062.81,1585623.20,3692967.16,9532118.80,1.716979e+08,9.83%,8.63%,7.70%


In [89]:
df_age_Ms = df_age_M[['M1+逾期率', 'M2+逾期率', 'M3+逾期率']]
df_ages_group = pd.merge(df_ages, df_age_Ms[:-1], on='age_group', how='outer')
df_ages_group[['出库', '出库', '买断价', '出库(移除已完成)', '出库(移除已完成)', '买断价(移除已完成)', 'M1+逾期率', 'M2+逾期率', 'M3+逾期率']]

,出库,出库,买断价,出库(移除已完成),出库(移除已完成),买断价(移除已完成),M1+逾期率,M2+逾期率,M3+逾期率
age_group,,,,,,,,,
"(0,22]",5573,5573,5.778826e+07,4293,4293,4.513889e+07,9.48%,8.47%,7.67%
"(22,30]",9974,9974,1.123855e+08,8951,8951,1.015255e+08,10.46%,9.26%,8.33%
"(30,40]",6312,6312,7.238519e+07,5858,5858,6.754909e+07,9.38%,7.94%,7.00%
"(40,55]",1583,1583,1.853075e+07,1494,1494,1.762011e+07,8.86%,7.97%,6.71%
[55+),0,0,0.000000e+00,0,0,0.000000e+00,nan%,nan%,nan%


# 地域分布

In [90]:
df[['order_number', 'delivery_province_name']]

df_province1 = pd.crosstab(df["下单月份"],df["delivery_province_name"],margins=True)

df['delivery_province_name'] = np.where(df['delivery_province_name']=='上海', '上海市', 
                                        np.where(df['delivery_province_name']=='北京', '北京市',
                                                np.where(df['delivery_province_name']=='天津', '天津市',
                                                        np.where(df['delivery_province_name']=='重庆', '重庆市', df['delivery_province_name']))))
province_list = ['广东省', '江苏省', '浙江省', '福建省', '山东省', '四川省', '河南省', '河北省', '安徽省', '云南省', '湖北省', '湖南省', '江西省', '陕西省', '上海市', '辽宁省', 
                '北京市', '重庆市', '贵州省', '广西壮族自治区', '山西省', '黑龙江省', '吉林省', '甘肃省', '天津市', '海南省']

df_province_M = M_yq(df, 'delivery_province_name')

df_province = df.groupby('delivery_province_name').agg({'是否出库': 'sum', 'new_actual_money': 'sum'})

df_province.rename(columns={'是否出库': '出库', 'new_actual_money': '买断价'}, inplace=True)

In [91]:
df_tc = df[df['status2']!='已完成']
df_province_tc = df_tc.groupby('delivery_province_name').agg({'是否出库': 'sum', 'new_actual_money': 'sum'})

df_province_tc.rename(columns={'是否出库': '出库(移除已完成)', 'new_actual_money': '买断价(移除已完成)'}, inplace=True)
df_provinces = pd.merge(df_province, df_province_tc, on='delivery_province_name', how='outer')

In [92]:
df_province_M

,M0,M1,M2,M3,M6,purchase_amount,M1+逾期率,M2+逾期率,M3+逾期率
delivery_province_name,,,,,,,,,
上海市,54334.09,48913.12,53056.97,108742.12,206912.70,5.286390e+06,7.90%,6.97%,5.97%
云南省,103195.11,106202.01,51294.33,166075.31,256588.60,5.699021e+06,10.18%,8.32%,7.42%
内蒙古自治区,61500.41,18637.30,33445.17,38154.20,64717.89,2.623654e+06,5.91%,5.20%,3.92%
北京市,58587.40,64749.12,45971.70,54895.36,139012.73,3.883592e+06,7.84%,6.18%,4.99%
吉林省,77871.60,31639.87,26952.93,76390.87,180742.79,3.172662e+06,9.95%,8.95%,8.10%
四川省,254086.64,107701.42,102437.85,232937.79,671783.67,9.851926e+06,11.32%,10.22%,9.18%
天津市,30497.15,18893.46,8617.78,37185.44,144312.31,1.761154e+06,11.87%,10.79%,10.31%
宁夏回族自治区,14551.10,23693.75,0.00,9758.16,56836.09,7.670200e+05,11.77%,8.68%,8.68%
安徽省,170650.09,68548.94,107254.38,130136.33,324217.83,7.404447e+06,8.51%,7.58%,6.14%


In [93]:
df_province_Ms = df_province_M[['M1+逾期率', 'M2+逾期率', 'M3+逾期率']]
df_provinces_group = pd.merge(df_provinces, df_province_Ms[:-1], on='delivery_province_name', how='outer')
df_provinces_group[['出库', '出库', '买断价', '出库(移除已完成)', '出库(移除已完成)', '买断价(移除已完成)', 'M1+逾期率', 'M2+逾期率', 'M3+逾期率']]

,出库,出库,买断价,出库(移除已完成),出库(移除已完成),买断价(移除已完成),M1+逾期率,M2+逾期率,M3+逾期率
delivery_province_name,,,,,,,,,
上海市,716,716,8066208.38,613,613,6934830.14,7.90%,6.97%,5.97%
云南省,788,788,8671598.36,674,674,7590308.85,10.18%,8.32%,7.42%
内蒙古自治区,351,351,3982290.99,307,307,3515128.01,5.91%,5.20%,3.92%
北京市,528,528,5912660.59,472,472,5303013.74,7.84%,6.18%,4.99%
吉林省,422,422,4834819.34,373,373,4309939.95,9.95%,8.95%,8.10%
四川省,1338,1338,14941193.05,1171,1171,13242298.63,11.32%,10.22%,9.18%
天津市,236,236,2680157.78,210,210,2405870.01,11.87%,10.79%,10.31%
宁夏回族自治区,105,105,1147267.61,90,90,977240.52,11.77%,8.68%,8.68%
安徽省,1002,1002,11311020.99,870,870,9887098.61,8.51%,7.58%,6.14%


# 单笔放款金额区间

In [94]:
# df[df['purchase_amount']==0]['order_number'].count()

In [95]:
# # 定义金额区间
# price_bins = [0, 3000, 5000, 10000, 20000, 30000]
# price_labels = ['(0, 3000)', '[3000,5000)', '[5000,10000)', '[10000,20000)', '[20000,30000)']

# # 使用 pd.cut 设置金额区间
# df['price_group'] = pd.cut(df['new_actual_money'], bins=price_bins, labels=price_labels, right=True, include_lowest=False)

# df[['order_number', 'contract_price', 'price_group']]
# df_price = pd.crosstab(df["下单月份"],df["price_group"],margins=True)
# df_price['(0, 3000)比例'] = df_price['(0, 3000)'] / df_price['All']
# df_price['(0, 3000)比例'] = df_price['(0, 3000)比例'].apply(lambda x: format(x, '.2%'))

# df_price['[3000,5000)比例'] = df_price['[3000,5000)'] / df_price['All']
# df_price['[3000,5000)比例'] = df_price['[3000,5000)比例'].apply(lambda x: format(x, '.2%'))

# df_price['[5000,10000)比例'] = df_price['[5000,10000)'] / df_price['All']
# df_price['[5000,10000)比例'] = df_price['[5000,10000)比例'].apply(lambda x: format(x, '.2%'))

# df_price['[10000,20000)比例'] = df_price['[10000,20000)'] / df_price['All']
# df_price['[10000,20000)比例'] = df_price['[10000,20000)比例'].apply(lambda x: format(x, '.2%'))

# df_price